In [1]:
#only type1 websites
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Read the XLSX file
xlsx_file = 'Input.xlsx'
df = pd.read_excel(xlsx_file)

# Extract the URLs from the Excel file
urls = df['URL'].tolist()

x=0
# Iterate over the URLs
for url in urls:
    x=x+1
    try:
        # Send a GET request to the website
        response = requests.get(url)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        divs=soup.find_all('div', class_='tdb-block-inner td-fix-index')
        article=[]
    
        for div in divs:
            temp=[]
            title=div.find('h1', class_='tdb-title-text')
            if title:
                article.append(title.text)
            paragraphs=div.find_all('p')
            for paragraph in paragraphs:
                para=paragraph.text
                if para:
                    temp.append(para)
            if len(temp)>0:
                article.append(temp)
            temp=[]
            
            
        #write the article to a text file
        if article:
            filename='ExtractedArticles/'+str(df['URL_ID'][x-1])+'.txt'
            with open(filename, 'w') as file:
                for i in article:
                    try:
                        file.write(i)
                        file.write('\n')
                    except TypeError as e:
                        for j in i:
                            file.write(j)
                            file.write('\n')

    except requests.exceptions.RequestException as e:
        print('Error accessing the website (serial number',x,'):', url)
        print('Exception:', e)


Error accessing the website (serial number 8 ): https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Exception: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Error accessing the website (serial number 21 ): https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Exception: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Error accessing the website (serial number 108 ): https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/
Exception: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/


In [2]:
#only type2 websites
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Read the XLSX file
xlsx_file = 'Input.xlsx'
df = pd.read_excel(xlsx_file)

# Extract the URLs from the Excel file
urls = df['URL'].tolist()

x=0
# Iterate over the URLs
for url in urls:
    x=x+1
    try:
        # Send a GET request to the website
        response = requests.get(url)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        divs=soup.find_all('div', class_='td-parallax-header')
        article=[]
    
        for div in divs:
            title=div.find('h1', class_='entry-title')
            if title:
                article.append(title.text)
        divs=soup.find_all('div', class_='td-post-content tagdiv-type')
        for div in divs:
            temp=[]
            paragraphs=div.find_all('p')
            for paragraph in paragraphs:
                para=paragraph.text
                if para:
                    temp.append(para)
            if len(temp)>0:
                article.append(temp)
            temp=[]


        #write the article to a text file
        if article:
            filename='ExtractedArticles/'+str(df['URL_ID'][x-1])+'.txt'
            with open(filename, 'w') as file:
                for i in article:
                    try:
                        file.write(i)
                        file.write('\n')
                    except TypeError as e:
                        for j in i:
                            file.write(j)
                            file.write('\n')
                            
    except requests.exceptions.RequestException as e:
        print('Error accessing the website (serial number',x,'):', url)
        print('Exception:', e)

Error accessing the website (serial number 8 ): https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Exception: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Error accessing the website (serial number 21 ): https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Exception: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
Error accessing the website (serial number 108 ): https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/
Exception: 404 Client Error: Not Found for url: https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/


In [7]:
#Text Analysis
import pandas as pd
import openpyxl
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
 
import re
def count_syllables(word):
    word = word.lower()
    if len(word) <= 3:
        return 1

    # Remove non-alphabetic characters
    word = re.sub(r'[^a-z]', '', word)

    # Handle exceptions like words ending with "es" or "ed"
    exceptions = ['es', 'ed']
    for exception in exceptions:
        if word.endswith(exception):
            word = word[:-len(exception)]
            break

    # Count syllables based on vowel patterns
    vowels = re.findall(r'[aeiouy]+', word)
    return max(1, len(vowels))

def is_personal_pronouns(word):
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    if pronounRegex.search(word):
        return 1
    else:
        return 0

def is_complex_word(word):
    syllable_count = count_syllables(word)
    return syllable_count > 2

import os

directory = 'StopWords'
stopwords = []
positive_words=[]
negative_words=[]

encodings = ['utf-8', 'latin-1', 'utf-16']  # Add more encodings if needed

# Iterate over files in the directory
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if filename.endswith('.txt') and os.path.isfile(file_path):
        # Try different encodings to read the file
        for encoding in encodings:
            try:
                with open(file_path, 'r', encoding=encoding) as file:
                    file_content = file.read().split()
                    for word in file_content:
                        stopwords.append(word)
                # Append the file content to the list
                
                break  # Break the loop if a successful encoding is found

            except UnicodeDecodeError:
                continue  # Try the next encoding if decoding fails

with open('MasterDictionary/negative-words.txt', 'r', encoding='latin-1') as file:
            file_content = file.read().split()
            for word in file_content:
                if word not in stopwords:
                    negative_words.append(word)

for encoding in encodings:
    try:
        with open('MasterDictionary/positive-words.txt', 'r') as file:
            file_content = file.read().split()
            for word in file_content:
                if word not in stopwords:
                    positive_words.append(word)
        break
    except UnicodeDecodeError:
        continue

output_file='output.xlsx'
wb_obj = openpyxl.load_workbook(output_file)
sheet_obj = wb_obj.active
row_index=1


punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
df = pd.read_excel(output_file)
url_id = df['URL_ID'].tolist()
for id in url_id:
    row_index=row_index+1
    filename='ExtractedArticles/'+str(id)+'.txt'
    try:
        with open(filename, 'r') as file:
            positive_score=0
            negative_score=0
            polarity_score=0
            subjectivity_score=0
            sentence_count=0
            avg_sentence_length=0
            complex_word_count=0
            percentage_complex_words=0
            fog_index=0
            avg_words_per_sentence=0
            syllable_count_per_word=0
            avg_word_length=0
            personal_pronouns=0

            data=file.read()
            tokens=word_tokenize(data)
            for word in tokens:
                if word in stopwords:
                    tokens.remove(word)
                if word == '.':
                    sentence_count=sentence_count+1
                if word in punc:
                    tokens.remove(word)
            for word in tokens:
                if is_personal_pronouns(word):
                    personal_pronouns=personal_pronouns+1
                avg_word_length=avg_word_length+len(word)
                syllable_count_per_word=syllable_count_per_word+count_syllables(word)
                if is_complex_word(word):
                    complex_word_count=complex_word_count+1
                if word in positive_words:
                    positive_score=positive_score+1
                elif word in negative_words:
                    negative_score=negative_score-1
                    negative_score=negative_score*(-1)
            polarity_score=(positive_score - negative_score)/((positive_score + negative_score)+0.000001)
            subjectivity_score=(positive_score + negative_score)/(len(tokens)+0.000001)
            avg_sentence_length=len(tokens)/sentence_count
            percentage_complex_words=complex_word_count/len(tokens)
            fog_index=0.4*(avg_sentence_length+percentage_complex_words)
            avg_words_per_sentence=avg_sentence_length
            avg_word_length=avg_word_length/len(tokens)
            sheet_obj.cell(row=row_index, column=3).value = positive_score
            sheet_obj.cell(row=row_index, column=4).value = negative_score
            sheet_obj.cell(row=row_index, column=5).value = polarity_score
            sheet_obj.cell(row=row_index, column=6).value = subjectivity_score
            sheet_obj.cell(row=row_index, column=7).value = avg_sentence_length
            sheet_obj.cell(row=row_index, column=8).value = percentage_complex_words
            sheet_obj.cell(row=row_index, column=9).value = fog_index
            sheet_obj.cell(row=row_index, column=10).value = avg_words_per_sentence
            sheet_obj.cell(row=row_index, column=11).value = complex_word_count
            sheet_obj.cell(row=row_index, column=12).value = len(tokens)
            sheet_obj.cell(row=row_index, column=13).value = syllable_count_per_word
            sheet_obj.cell(row=row_index, column=14).value = personal_pronouns
            sheet_obj.cell(row=row_index, column=15).value = avg_word_length

    except FileNotFoundError as e:
        print('File not found:', filename)
        print('Exception:', e)
    wb_obj.save(output_file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


File not found: ExtractedArticles/44.txt
Exception: [Errno 2] No such file or directory: 'ExtractedArticles/44.txt'
File not found: ExtractedArticles/57.txt
Exception: [Errno 2] No such file or directory: 'ExtractedArticles/57.txt'
File not found: ExtractedArticles/144.txt
Exception: [Errno 2] No such file or directory: 'ExtractedArticles/144.txt'
